In [56]:
import pandas as pd
import os
import glob
import numpy as np
# _ = os.stat(f'{os.path.expanduser("~")}/experiments/').st_mtime    # to refresh the cache

# metric_files = glob.glob(f'{os.path.expanduser("~")}/experiments/*/logfile*')
metric_files = glob.glob(f'{os.path.expanduser("~")}/experiments/dev/*/logfile')
minus_metric_files = []
# minus_metric_files += glob.glob(f'{os.path.expanduser("~")}/experiments/*5000/logfile*')
# minus_metric_files += glob.glob(f'{os.path.expanduser("~")}/experiments/dev/logfile*')
# minus_metric_files += glob.glob(f'{os.path.expanduser("~")}/experiments/ours/logfile*')

metric_files = [x for x in metric_files if x not in minus_metric_files]

all_metrics = []

# include args
include_args = ['dataset']
for i in range(len(metric_files)):
    file = metric_files[i]
    opened_file = open(file, 'r')

    accum_info = {}

    for line in opened_file.readlines():

        args = np.load(os.path.dirname(file) + '/args.npy', allow_pickle=True).item()

        # if 'dataset' in line:
        #     dataset = line.split('=')[-1].strip()
        #     dataset = dataset.capitalize()

        if 'exp_name' in line:
            exp_name = line.split('=')[-1].strip()

        if 'EPE3D' in line:
            epe3d = float(line.split(' ')[-1])

        if 'acc3d_strict' in line:
            acc3d_strict = float(line.split(' ')[-1])

        if 'acc3d_relax' in line:
            acc3d_relax = float(line.split(' ')[-1])

        if 'angle_error' in line:
            angle_error = float(line.split(' ')[-1])

        if 'outlier' in line:
            outlier = float(line.split(' ')[-1])

        if 'solver_time' in line:
            solver_time = float(line.split(' ')[-1])


    accum_info = {"exp_name" : exp_name,
                  "epe3d" : epe3d,
                  "acc3d_strict" : acc3d_strict,
                  "acc3d_relax" : acc3d_relax,
                  "angle_error" : angle_error,
                  "outlier" : outlier,
                  "solver_time" : solver_time}

    for includes in include_args:
        accum_info[includes] = getattr(args, includes)

    all_metrics.append(accum_info)

    opened_file.close()

metric_df = pd.DataFrame(all_metrics)


### Works, highlights  rows
# highest_epe = metric_df['epe3d'].argmax()
# print(highest_epe)
# subsets = pd.IndexSlice[highest_epe, :]

# metric_df.style.applymap(lambda x: "background-color: green", subset=subsets)
metric_df
# todo search for metrics you want, commands, plotting

,exp_name,epe3d,acc3d_strict,acc3d_relax,angle_error,outlier,solver_time,dataset
0,dev,0.245842,0.038680,0.183696,0.125528,0.912334,1.982607,argoverse
1,dev,0.246491,0.000000,0.120403,0.146714,0.967503,2.116008,argoverse
2,dev,0.336021,0.036059,0.065274,0.107339,0.952435,2.132563,argoverse
3,dev,0.280821,0.019770,0.088286,0.131208,0.944291,2.035323,argoverse
4,dev,0.254855,0.049906,0.127106,0.099550,0.925141,2.140059,argoverse
5,dev,0.270263,0.030296,0.128287,0.110837,0.913795,2.178515,argoverse
6,dev,0.177557,0.115800,0.451655,0.128515,0.735402,2.083488,argoverse
7,dev,0.298591,0.028095,0.091127,0.143512,0.945912,2.150250,argoverse
8,dev,0.298591,0.028095,0.091127,0.143512,0.945912,2.150250,argoverse
9,dev,0.231836,0.036479,0.137732,0.106635,0.914055,2.147244,argoverse


In [6]:
from data.PATHS import TMP_VIS_PATH
import matplotlib.pyplot as plt
plt.close()
plt.rcParams["figure.figsize"] = (18,10)
plt.title("Model performance")
plt.grid(True)
plt.xlabel("Time Complexity (s)")
plt.ylabel("EPE3D [m]")

# annotate points
for i in range(len(metric_df)):
    exp_name = metric_df['exp_name'][i]
    if 'ours' in exp_name or 'visibility' in exp_name or 'VIS' in exp_name or 'forward' in exp_name or "FF" in exp_name:
        color = 'r'
    else:
        color = 'b'

    # plot marker
    plt.plot(metric_df['solver_time'][i], metric_df['epe3d'][i], f'{color}*', markersize=20)

                    # Name of the point                         # Coordinates
    plt.annotate(exp_name.replace('-ours','(ours)'), (metric_df['solver_time'][i], metric_df['epe3d'][i]), fontsize=15)

plt.savefig(TMP_VIS_PATH + '/gif.png')


In [39]:
from vis.mayavi_interactive import visualize_flow_frame


[{'args': array(Namespace(dataset='argoverse', data_path='/home.dokt/vacekpa2/data/', exp_name='dev', baseline='NSF', gpu='0', verbose=0, max_range=35, per_sample_init=1, store_inference=1, runs=1, dev=1, vis=0, l_ch=1.0, l_ff=1, l_dt=0, l_sm=1, l_vsm=0, normals_K=4, KNN_max_radius=1.5, smooth_K=4, pc2_smoothness=0, l_ch_bothways=1, l_dt_gridfactor=10, lr=0.008, iters=10, early_patience=10, early_min_delta=0.001, use_case='local minima', expectation='beat baseline', hypothesis='beat baseline'),
        dtype=object)},
 {'args': array(Namespace(dataset='argoverse', data_path='/home.dokt/vacekpa2/data/', exp_name='dev', baseline='NSF', gpu='0', verbose=0, max_range=35, per_sample_init=1, store_inference=1, runs=1, dev=1, vis=0, l_ch=1.0, l_ff=1, l_dt=0, l_sm=1, l_vsm=0, normals_K=4, KNN_max_radius=1.5, smooth_K=4, pc2_smoothness=0, l_ch_bothways=1, l_dt_gridfactor=10, lr=0.008, iters=10, early_patience=10, early_min_delta=0.001, use_case='local minima', expectation='beat baseline', hypot